## Задание по программированию: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.
Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat.

1. Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас.
2. С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [7]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift, estimate_bandwidth

In [8]:
data = pd.read_csv("checkins.dat", sep='|', header=None, skiprows=1,
                   low_memory = False, skipinitialspace=True,
                   names=['id','user_id','venue_id','latitude','longitude','created_at'])
data.dropna(subset=['latitude', 'longitude'], inplace = True)
data.reset_index(inplace=True, drop=True)

In [9]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396634 entries, 0 to 396633
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          396634 non-null  object 
 1   user_id     396634 non-null  float64
 2   venue_id    396634 non-null  float64
 3   latitude    396634 non-null  float64
 4   longitude   396634 non-null  float64
 5   created_at  396634 non-null  object 
dtypes: float64(4), object(2)
memory usage: 18.2+ MB


4. Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [22]:
ms = MeanShift(bandwidth=0.1, n_jobs=-1)


In [23]:
%%time
ms.fit(data.loc[:100000, ['latitude','longitude']])

Wall time: 1min 31s


MeanShift(bandwidth=0.1, n_jobs=-1)

In [24]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_

5. Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [25]:
cluster_centers = pd.DataFrame(ms.cluster_centers_[pd.Series(labels).value_counts() > 15],
    columns=['latitude','longitude'])

In [26]:
cluster_centers

,latitude,longitude
0,40.717716,-73.991835
1,33.449438,-112.002140
2,33.446380,-111.901888
3,41.878244,-87.629843
4,37.688682,-122.409330
...,...,...
587,39.202993,-84.404975
588,38.041182,-122.150053
589,37.270702,-76.707457
590,37.204643,-80.412689


6. Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [27]:
offices = pd.DataFrame(np.array([[33.751277, -118.188740],\
        [25.867736, -80.324116],\
        [51.503016, -0.075479],\
        [52.378894, 4.885084],\
        [39.366487, 117.036146],\
        [-33.868457, 151.205134]]),
        columns=['latitude','longitude'])

7. Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [28]:
for i in range(offices.shape[0]):
    cluster_centers[f"dist_to_{i}"] = np.linalg.norm(cluster_centers.loc[:, ['latitude','longitude']] - offices.values[i], axis=1).reshape(-1, 1)


In [29]:
cluster_centers.head()

,latitude,longitude,dist_to_0,dist_to_1,dist_to_2,dist_to_3,dist_to_4,dist_to_5
0,40.717716,-73.991835,44.742571,16.143720,74.699066,79.734255,191.032760,237.227259
1,33.449438,-112.002140,6.193959,32.572679,113.373317,118.410081,229.114704,271.679532
2,33.446380,-111.901888,6.294241,32.474475,113.274833,118.311609,229.014565,271.581649
3,41.878244,-87.629843,31.621097,17.598580,88.081797,93.108944,204.681402,250.558794
4,37.688682,-122.409330,5.772048,43.713842,123.111360,128.139261,239.451355,282.816724


In [30]:
cl_cen = cluster_centers.copy()

In [31]:
ind_dist = []
for j in range(20):
    ind = []
    for i in range(6):
        min_ind = cl_cen[f"dist_to_{i}"].idxmin()
        min_value = cl_cen[f"dist_to_{i}"].min()
        ind_dist.append([min_ind, min_value])
        ind.append(min_ind)
    
    cl_cen.drop(ind, inplace=True)

In [34]:
ind_dist_sort = sorted(ind_dist, key=lambda a_entry: a_entry[1])
ind_dist_sort[0:20] # 20 ближайших к офисам центров кластеров 

[[420, 0.007834758163107856],
 [370, 0.009353316185992226],
 [419, 0.02267406615838222],
 [58, 0.05005829482278787],
 [51, 0.07084773242717578],
 [29, 0.13410903336184654],
 [167, 0.16740596425035667],
 [92, 0.18887596060185083],
 [87, 0.19577945647763628],
 [42, 0.21181053682436798],
 [291, 0.22223329073179776],
 [320, 0.27130075950667704],
 [119, 0.2949788868004569],
 [55, 0.3022701186924605],
 [27, 0.30473050307840693],
 [11, 0.3148837903362732],
 [32, 0.3388104702511318],
 [159, 0.3408456533220572],
 [17, 0.37868750125029754],
 [47, 0.3867062248427277]]

In [35]:
with open('c3w1e1_submission.txt', 'w') as f:
    f.write(' '.join(str(i) for i in cluster_centers.loc[ind_dist_sort[0][0], ['latitude', 'longitude']].values))